# Persiapan Modul dan File

In [ ]:
!pip install keras-tuner --quiet

In [ ]:
# Mengimpor libraries umum yang sering digunakan
import os, shutil
import zipfile
import random
from random import sample
import shutil
from shutil import copyfile
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm as tq

# Mengimpor libraries untuk visualisasi
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread

# Mengimpor libraries untuk pemrosesan data gambar
import cv2
from PIL import Image
import skimage
from skimage import io
from skimage.transform import resize
from skimage.transform import rotate, AffineTransform, warp
from skimage import img_as_ubyte
from skimage.exposure import adjust_gamma
from skimage.util import random_noise

# Mengimpor libraries untuk pembuatan dan evaluasi model
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from tensorflow.keras import Model, layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import InputLayer, Conv2D, SeparableConv2D, MaxPooling2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# Mengabaikan peringatan
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import os
#folder_path = '/content/drive/MyDrive/drunk_detection_with_boxes'
#print(os.path.exists(folder_path))  # Harusnya True
#print(os.listdir(folder_path))
#print(os.listdir('/content/drive/MyDrive/drunk_detection_with_boxes/content'))


False


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/drunk_detection_with_boxes'

In [ ]:
# Copy seluruh folder dataset ke local storage
#!cp -r /content/drive/MyDrive/drunk_detection_with_boxes/content /content/

# Verifikasi copy berhasil
#!ls -la /content/drunk_detection_with_boxes_1/


total 672
drwx------ 5 root root   4096 Jun 11 05:58 .
drwxr-xr-x 1 root root   4096 Jun 11 06:08 ..
drwx------ 2 root root  73728 Jun 11 05:57 test
drwx------ 2 root root 466944 Jun 11 06:00 train
drwx------ 2 root root 139264 Jun 11 05:58 valid


# load data

In [ ]:
df_test=pd.read_csv('test_classes.csv')
df_train=pd.read_csv('train_classes.csv')
df_valid=pd.read_csv('valid_classes.csv')

In [ ]:
df_test.head()

,filename,drunk,sober
0,Drunkface505B3_jpg.rf.07914aae708986df3dce4037...,1,0
1,img_6-189-_jpg.rf.057c348f8e4737d67eae62215e5a...,0,1
2,img_12-82-_jpg.rf.03fa2c65f298df2f64e4e5f5d61f...,0,1
3,img_338_jpg.rf.092180962bf27c97d7f182277044d57...,0,1
4,Drunkface252B3_jpg.rf.089fb50b4f4020dc3f50169b...,1,0


In [ ]:
df_train.head()

,filename,drunk,sober
0,img_9-50-_jpg.rf.c08cec1598c314a3dd900e0251f91...,0,1
1,Drunkface266B3_jpg.rf.c07177a92cd7dff0a90712c3...,1,0
2,Drunkface393B3_jpg.rf.c081ef8d61f54710d4743eb1...,1,0
3,img_5-107-_jpg.rf.c0c68f3f97c30f5a0d640d7216fb...,0,1
4,img_5-38-_jpg.rf.c0667a246f8d2c4578ee005e90885...,0,1


In [ ]:
df_valid.head()

,filename,drunk,sober
0,img_9-113-_jpg.rf.00252b97d20016e93776d5475867...,0,1
1,Drunkface350B_jpg.rf.00ab819aed7a4fd274b44c1b2...,1,0
2,img_20-28-_jpg.rf.0226a99d81d9157bbacd38486c2e...,0,1
3,img_8-114-_jpg.rf.00dd6ed8b05bff9941ed44ed5f9c...,0,1
4,img_3-131-_jpg.rf.00851732c28ee88efa4ca3614c1f...,0,1


# Persiapan

## Penandaan muka dengan face recognition

dilakukan penanda muka menggunakan face recognition sehingga bisa ditandai bagian yang harus dipelajari oleh machine learning.

In [ ]:
# Mount Google Drive dulu
from google.colab import drive
drive.mount('/content/drive')

# Copy seluruh folder dataset ke local storage
!cp -r /content/drive/MyDrive/drunk_detection /content/

# Verifikasi copy berhasil
!ls -la /content/drunk_detection/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/content/drive/MyDrive/drunk_detection': No such file or directory
ls: cannot access '/content/drunk_detection/': No such file or directory


In [ ]:
import os

base_output = '/content/drunk_detection_with_boxes'
for split in ['train', 'valid', 'test']:
    os.makedirs(os.path.join(base_output, split), exist_ok=True)


In [ ]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=0e01a2274faf758fc33beb19e0d4220e18731727f4bb61a5499da03d4cf27f44
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [ ]:
import face_recognition
import cv2
from tqdm import tqdm

def process_and_save(df, input_dir, output_dir):
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        filename = row['filename']
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)
        try:
            image = face_recognition.load_image_file(input_path)
            face_locations = face_recognition.face_locations(image)
            image_bgr = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            for (top, right, bottom, left) in face_locations:
                cv2.rectangle(image_bgr, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.imwrite(output_path, image_bgr)
        except Exception as e:
            print(f"Error processing {filename}: {e}")


In [ ]:
process_and_save(df_train, '/content/drunk_detection/train', f'{base_output}/train')
process_and_save(df_valid, '/content/drunk_detection/valid', f'{base_output}/valid')
process_and_save(df_test,  '/content/drunk_detection/test',  f'{base_output}/test')


100%|██████████| 734/734 [03:21<00:00,  3.65it/s]


In [ ]:
!zip -r /content/drunk_detection_with_boxes.zip /content/drunk_detection_with_boxes
from google.colab import files
files.download('/content/drunk_detection_with_boxes.zip')


Streaming output truncated to the last 5000 lines.
  adding: content/drunk_detection_with_boxes/train/img_10-19-_jpg.rf.9fec65bd8a651d370e0968e05f3a41d1.jpg (deflated 4%)
  adding: content/drunk_detection_with_boxes/train/Drunkface160B2_jpg.rf.dcd2ef5ee6f05ae23616f619c40f9114.jpg (deflated 6%)
  adding: content/drunk_detection_with_boxes/train/img_5-12-_jpg.rf.c4e9178cfe36801c818c24c25809ba06.jpg (deflated 4%)
  adding: content/drunk_detection_with_boxes/train/Drunkface272B2_jpg.rf.57965f40512793ed9b78b168df8e0db1.jpg (deflated 5%)
  adding: content/drunk_detection_with_boxes/train/Drunkface278B1_jpg.rf.7ea4432c8a6fe223eb4dacfd067efabe.jpg (deflated 6%)
  adding: content/drunk_detection_with_boxes/train/img_16-23-_jpg.rf.5211078d7c33ccf0a7f2ed0dad1d23c9.jpg (deflated 5%)
  adding: content/drunk_detection_with_boxes/train/img_23-84-_jpg.rf.1d35b2329b4bc1aa9ee815353c60345c.jpg (deflated 4%)
  adding: content/drunk_detection_with_boxes/train/Drunkface621B1_jpg.rf.cc6d3e0a4fc00e829a3cfd7fc

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Copy seluruh folder dataset ke local storage
!cp -r /content/drive/MyDrive/drunk_detection_with_boxes_1 /content/

# Verifikasi copy berhasil
!ls -la /content/drunk_detection_with_boxes_1/


total 12
drwx------ 2 root root 4096 Jun  1 05:23 test
drwx------ 2 root root 4096 Jun  1 05:23 train
drwx------ 2 root root 4096 Jun  1 05:23 valid


In [ ]:
# menyiapkan data agar siap untuk digunakan di ML
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory='/content/drunk_detection_with_boxes_1/train/',  # <-- Ubah ke local path
    x_col='filename',
    y_col=['drunk', 'sober'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=True
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=df_valid,
    directory='/content/drunk_detection_with_boxes_1/valid/',  # <-- Ubah ke local path
    x_col='filename',
    y_col=['drunk', 'sober'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory='/content/drunk_detection_with_boxes_1/test/',  # <-- Ubah ke local path
    x_col='filename',
    y_col=['drunk', 'sober'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=False
)


Found 5153 validated image filenames.
Found 1478 validated image filenames.
Found 734 validated image filenames.


## Penanda muka dengan Haar cascade

dicoba menggunakan Haar Cascade untuk face recognition

In [ ]:

!wget "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml" -O haarcascade_frontalface_default.xml
!wget "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml" -O haarcascade_eye.xml


--2025-06-03 07:02:11--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.01s   

2025-06-03 07:02:12 (90.0 MB/s) - ‘haarcascade_frontalface_default.xml’ saved [930127/930127]

--2025-06-03 07:02:12--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_eye.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... c

In [ ]:
import cv2
import os

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')


In [ ]:
def detect_faces_eyes(img_path, output_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Wajah: biru
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]

        # Deteksi mata
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 255), 2)  # Mata: kuning

    cv2.imwrite(output_path, img)


In [ ]:
def process_folder(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, filename)
            detect_faces_eyes(input_path, output_path)


In [ ]:
process_folder('/content/drunk_detection/train', '/content/drunk_detection_haar/train')
process_folder('/content/drunk_detection/valid', '/content/drunk_detection_haar/valid')
process_folder('/content/drunk_detection/test', '/content/drunk_detection_haar/test')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    directory='/content/drunk_detection_haar/train/',  # <-- Ubah ke local path
    x_col='filename',
    y_col=['drunk', 'sober'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=True
)

valid_generator = valid_datagen.flow_from_dataframe(
    dataframe=df_valid,
    directory='/content/drunk_detection_haar/valid/',  # <-- Ubah ke local path
    x_col='filename',
    y_col=['drunk', 'sober'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=False
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    directory='/content/drunk_detection_haar/test/',  # <-- Ubah ke local path
    x_col='filename',
    y_col=['drunk', 'sober'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='raw',
    shuffle=False
)


Found 5153 validated image filenames.
Found 1478 validated image filenames.
Found 734 validated image filenames.


# Training

## Model CNN

In [ ]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),
    BatchNormalization(),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    BatchNormalization(),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(2, activation='softmax')  # Output 2 neuron, karena label [drunk, sober]
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # karena label one-hot
    metrics=['accuracy']
)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    callbacks=[]  # bisa tambahkan EarlyStopping jika mau
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 2722s 17s/step - accuracy: 0.7377 - loss: 4.1607 - val_accuracy: 0.3471 - val_loss: 5.2051
Epoch 2/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 40s 245ms/step - accuracy: 0.8490 - loss: 0.9050 - val_accuracy: 0.5798 - val_loss: 6.2175
Epoch 3/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 38s 234ms/step - accuracy: 0.8611 - loss: 1.7178 - val_accuracy: 0.7199 - val_loss: 6.8666
Epoch 4/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 38s 233ms/step - accuracy: 0.8176 - loss: 4.1103 - val_accuracy: 0.6949 - val_loss: 7.9308
Epoch 5/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 40s 245ms/step - accuracy: 0.8254 - loss: 8.8240 - val_accuracy: 0.7666 - val_loss: 8.3561
Epoch 6/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 38s 233ms/step - accuracy: 0.8380 - loss: 8.5816 - val_accuracy: 0.6712 - val_loss: 4.8305
Epoch 7/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 40s 250ms/step - accuracy: 0.8273 - loss: 10.5901 - val_accuracy: 0.6252 - val_loss: 42.7446
Epoch 8/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 37s 231ms/step - accuracy: 0.8130 - loss:

## tuning cnn

In [ ]:
!pip install keras-tuner --quiet

import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
def build_cnn_model(hp):
    model = Sequential()

    # Input layer
    model.add(Conv2D(
        filters=hp.Int('conv_1_filter', min_value=16, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
        activation='relu',
        input_shape=(224, 224, 3)
    ))
    model.add(MaxPooling2D(hp.Choice('pool_1_size', values=[2, 3])))

    # Tuning: apakah pakai BatchNorm di layer pertama
    if hp.Boolean('bn_1'):
        model.add(BatchNormalization())

    # Tuning: jumlah conv layer tambahan (1-3 layer)
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        model.add(Conv2D(
            filters=hp.Int(f'conv_{i+2}_filter', min_value=32, max_value=256, step=32),
            kernel_size=hp.Choice(f'conv_{i+2}_kernel', values=[3, 5]),
            activation='relu'
        ))
        model.add(MaxPooling2D(hp.Choice(f'pool_{i+2}_size', values=[2, 3])))

        # Tuning: BatchNorm untuk setiap layer
        if hp.Boolean(f'bn_{i+2}'):
            model.add(BatchNormalization())

    # Flatten
    model.add(Flatten())

    # Tuning: jumlah Dense layer (1-2 layer)
    for i in range(hp.Int('num_dense_layers', 1, 2)):
        model.add(Dense(
            units=hp.Int(f'dense_{i+1}_units', min_value=32, max_value=256, step=32),
            activation='relu'
        ))
        # Tuning: Dropout
        model.add(Dropout(hp.Float(f'dropout_{i+1}', 0.2, 0.6, step=0.1)))

    # Output layer
    model.add(Dense(2, activation='softmax'))

    # Tuning: Learning rate
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model


In [ ]:
tuner = kt.RandomSearch(
    build_cnn_model,
    objective='val_accuracy',
    max_trials=15,  # Banyak kombinasi yang dicoba
    executions_per_trial=1,
    directory='cnn_tuner_dir',
    project_name='drunk_sober_cnn_tuning'
)

# Lihat search space
tuner.search_space_summary()


Search space summary
Default search space size: 13
conv_1_filter (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
conv_1_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
pool_1_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
bn_1 (Boolean)
{'default': False, 'conditions': []}
num_conv_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': 'linear'}
conv_2_filter (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
conv_2_kernel (Choice)
{'default': 3, 'conditions': [], 'values': [3, 5], 'ordered': True}
pool_2_size (Choice)
{'default': 2, 'conditions': [], 'values': [2, 3], 'ordered': True}
bn_2 (Boolean)
{'default': False, 'conditions': []}
num_dense_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling'

In [ ]:
callbacks = [
    EarlyStopping(patience=3, restore_best_weights=True)
]

tuner.search(
    train_generator,
    validation_data=valid_generator,
    epochs=15,
    callbacks=callbacks
)


Trial 15 Complete [00h 04m 47s]
val_accuracy: 0.7496616840362549

Best val_accuracy So Far: 0.9235453605651855
Total elapsed time: 01h 22m 00s


In [ ]:
# Ambil model terbaik
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparams = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters:")
print(best_hyperparams.values)

# Simpan model terbaik
best_model.save('/content/drive/MyDrive/best_cnn_tuned.h5')

# Lihat summary model terbaik
best_model.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Best hyperparameters:
{'conv_1_filter': 128, 'conv_1_kernel': 5, 'pool_1_size': 2, 'bn_1': True, 'num_conv_layers': 2, 'conv_2_filter': 192, 'conv_2_kernel': 3, 'pool_2_size': 3, 'bn_2': True, 'num_dense_layers': 2, 'dense_1_units': 160, 'dropout_1': 0.2, 'learning_rate': 0.0001, 'conv_3_filter': 128, 'conv_3_kernel': 3, 'pool_3_size': 2, 'bn_3': True, 'dense_2_units': 96, 'dropout_2': 0.30000000000000004}


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 220, 220, 128)  │         9,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 110, 110, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 110, 110, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 108, 108, 192)  │       221,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 192)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 36, 36, 192)    │           768 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │       221,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 17, 17, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36992)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 160)            │     5,918,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 160)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 96)             │        15,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 96)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           194 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,388,738 (24.37 MB)

 Trainable params: 6,387,842 (24.37 MB)

 Non-trainable params: 896 (3.50 KB)

In [ ]:
import joblib
joblib.dump(best_model, '/content/drive/MyDrive/best_modelCNN.joblib')

['/content/drive/MyDrive/best_modelCNN.joblib']

# Evaluasi model terbaik CNN

In [ ]:
# Evaluasi pada data test
score = best_model.evaluate(test_generator)
print(f"Test accuracy: {score[1]:.4f}")

# Train lebih lama dengan hyperparameter terbaik
best_model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,
    callbacks=[EarlyStopping(patience=5, restore_best_weights=True)]
)


23/23 ━━━━━━━━━━━━━━━━━━━━ 300s 14s/step - accuracy: 0.9151 - loss: 0.2833
Test accuracy: 0.9060
Epoch 1/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 53s 282ms/step - accuracy: 0.9664 - loss: 0.1073 - val_accuracy: 0.9208 - val_loss: 0.3547
Epoch 2/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 42s 256ms/step - accuracy: 0.9782 - loss: 0.1206 - val_accuracy: 0.8938 - val_loss: 0.4769
Epoch 3/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 46s 281ms/step - accuracy: 0.9628 - loss: 0.1835 - val_accuracy: 0.9080 - val_loss: 0.4545
Epoch 4/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 43s 262ms/step - accuracy: 0.9619 - loss: 0.2312 - val_accuracy: 0.9175 - val_loss: 0.5071
Epoch 5/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 41s 250ms/step - accuracy: 0.9588 - loss: 0.2633 - val_accuracy: 0.9290 - val_loss: 0.4490
Epoch 6/30
162/162 ━━━━━━━━━━━━━━━━━━━━ 43s 264ms/step - accuracy: 0.9527 - loss: 0.5103 - val_accuracy: 0.8627 - val_loss: 1.1684


# Transfer learning

In [ ]:


# Parameter
input_shape = (224, 224, 3)
num_classes = 2  # drunk, sober

# Backbone EfficientNetB0
base_model = EfficientNetB0(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape
)
base_model.trainable = False  # Freeze dulu layer backbone

# Bangun model
model_transfer = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [ ]:
model_transfer.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
callbacks = [
    EarlyStopping(patience=8, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True)
]
history = model_transfer.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.6392 - loss: 0.6608

162/162 ━━━━━━━━━━━━━━━━━━━━ 67s 274ms/step - accuracy: 0.6393 - loss: 0.6607 - val_accuracy: 0.6529 - val_loss: 0.6460
Epoch 2/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 24s 145ms/step - accuracy: 0.6565 - loss: 0.6461 - val_accuracy: 0.6529 - val_loss: 0.6474
Epoch 3/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 24s 146ms/step - accuracy: 0.6537 - loss: 0.6514 - val_accuracy: 0.6529 - val_loss: 0.6464
Epoch 4/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.6379 - loss: 0.6580

162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 152ms/step - accuracy: 0.6379 - loss: 0.6580 - val_accuracy: 0.6529 - val_loss: 0.6459
Epoch 5/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 40s 147ms/step - accuracy: 0.6493 - loss: 0.6522 - val_accuracy: 0.6529 - val_loss: 0.6462
Epoch 6/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 23s 144ms/step - accuracy: 0.6558 - loss: 0.6474 - val_accuracy: 0.6529 - val_loss: 0.6471
Epoch 7/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 42s 151ms/step - accuracy: 0.6564 - loss: 0.6452 - val_accuracy: 0.6529 - val_loss: 0.6491
Epoch 8/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 29s 178ms/step - accuracy: 0.6614 - loss: 0.6503 - val_accuracy: 0.6529 - val_loss: 0.6460
Epoch 9/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.6549 - loss: 0.6505

162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 156ms/step - accuracy: 0.6548 - loss: 0.6505 - val_accuracy: 0.6529 - val_loss: 0.6458
Epoch 10/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 29s 177ms/step - accuracy: 0.6583 - loss: 0.6435 - val_accuracy: 0.6529 - val_loss: 0.6464
Epoch 11/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 24s 147ms/step - accuracy: 0.6585 - loss: 0.6484 - val_accuracy: 0.6529 - val_loss: 0.6461
Epoch 12/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 23s 144ms/step - accuracy: 0.6503 - loss: 0.6546 - val_accuracy: 0.6529 - val_loss: 0.6466
Epoch 13/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 42s 150ms/step - accuracy: 0.6543 - loss: 0.6512 - val_accuracy: 0.6529 - val_loss: 0.6461
Epoch 14/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 28s 176ms/step - accuracy: 0.6421 - loss: 0.6568 - val_accuracy: 0.6529 - val_loss: 0.6463
Epoch 15/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 155ms/step - accuracy: 0.6525 - loss: 0.6500 - val_accuracy: 0.6529 - val_loss: 0.6465
Epoch 16/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 155ms/step - accuracy: 0.6571 - loss: 0.646

## Fine tuning EfficientNetB0

In [ ]:
# Fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-10]:
    layer.trainable = False

model_transfer.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_fine = model_transfer.fit(  # <-- Tambahkan .fit()
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    callbacks=callbacks
)


Epoch 1/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 77s 321ms/step - accuracy: 0.5836 - loss: 0.6918 - val_accuracy: 0.6529 - val_loss: 0.6590
Epoch 2/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 24s 151ms/step - accuracy: 0.6213 - loss: 0.6778 - val_accuracy: 0.6529 - val_loss: 0.6660
Epoch 3/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 154ms/step - accuracy: 0.6341 - loss: 0.6666 - val_accuracy: 0.6529 - val_loss: 0.6523
Epoch 4/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 23s 141ms/step - accuracy: 0.6472 - loss: 0.6555 - val_accuracy: 0.6529 - val_loss: 0.6490
Epoch 5/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 152ms/step - accuracy: 0.6350 - loss: 0.6568 - val_accuracy: 0.6529 - val_loss: 0.6467
Epoch 6/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 24s 149ms/step - accuracy: 0.6607 - loss: 0.6482 - val_accuracy: 0.6529 - val_loss: 0.6482
Epoch 7/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 24s 147ms/step - accuracy: 0.6453 - loss: 0.6565 - val_accuracy: 0.6529 - val_loss: 0.6472
Epoch 8/10
162/162 ━━━━━━━━━━━━━━━━━━━━ 29s 178ms/step - accuracy: 0.6405 - loss: 0

162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 158ms/step - accuracy: 0.6429 - loss: 0.6564 - val_accuracy: 0.6529 - val_loss: 0.6455


# Transfel Learning dengan Resnet50

In [ ]:

input_shape = (224, 224, 3)
num_classes = 2  # drunk, sober

# Backbone ResNet50
base_model_res = ResNet50(include_top=False,
                      weights='imagenet',
                      input_shape=input_shape)
base_model_res.trainable = False  # Freeze dulu layer backbone


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
model_res = models.Sequential([
    base_model_res,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])


In [ ]:
model_res.compile(optimizer=Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model_res.h5', save_best_only=True)
]


In [ ]:
history = model_res.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20,
    callbacks=callbacks
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.6094 - loss: 0.7364

162/162 ━━━━━━━━━━━━━━━━━━━━ 55s 247ms/step - accuracy: 0.6096 - loss: 0.7360 - val_accuracy: 0.6529 - val_loss: 0.6316
Epoch 2/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.6591 - loss: 0.6307

162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 160ms/step - accuracy: 0.6591 - loss: 0.6307 - val_accuracy: 0.6529 - val_loss: 0.6223
Epoch 3/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.6484 - loss: 0.6317

162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 159ms/step - accuracy: 0.6484 - loss: 0.6317 - val_accuracy: 0.6529 - val_loss: 0.6014
Epoch 4/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.6506 - loss: 0.6115

...

162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 157ms/step - accuracy: 0.6879 - loss: 0.5635 - val_accuracy: 0.7402 - val_loss: 0.5511
Epoch 8/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 40s 151ms/step - accuracy: 0.6865 - loss: 0.5837 - val_accuracy: 0.7023 - val_loss: 0.5569
Epoch 9/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.6794 - loss: 0.5856

162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 162ms/step - accuracy: 0.6794 - loss: 0.5855 - val_accuracy: 0.6793 - val_loss: 0.5346
Epoch 10/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 151ms/step - accuracy: 0.6752 - loss: 0.5823 - val_accuracy: 0.6529 - val_loss: 0.5764
Epoch 11/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 156ms/step - accuracy: 0.6328 - loss: 0.5970 - val_accuracy: 0.6529 - val_loss: 0.5493
Epoch 12/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 45s 178ms/step - accuracy: 0.6485 - loss: 0.5853 - val_accuracy: 0.7375 - val_loss: 0.5403
Epoch 13/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 158ms/step - accuracy: 0.6586 - loss: 0.5737 - val_accuracy: 0.7673 - val_loss: 0.5444
Epoch 14/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.6548 - loss: 0.5685

162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 156ms/step - accuracy: 0.6548 - loss: 0.5685 - val_accuracy: 0.6915 - val_loss: 0.5251
Epoch 15/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 0.6711 - loss: 0.5603

162/162 ━━━━━━━━━━━━━━━━━━━━ 30s 183ms/step - accuracy: 0.6712 - loss: 0.5604 - val_accuracy: 0.6529 - val_loss: 0.5173
Epoch 16/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 152ms/step - accuracy: 0.6797 - loss: 0.5597 - val_accuracy: 0.7876 - val_loss: 0.5410
Epoch 17/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.6802 - loss: 0.5583

162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 158ms/step - accuracy: 0.6802 - loss: 0.5583 - val_accuracy: 0.7564 - val_loss: 0.5060
Epoch 18/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 29s 180ms/step - accuracy: 0.6546 - loss: 0.5759 - val_accuracy: 0.6529 - val_loss: 0.5196
Epoch 19/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 156ms/step - accuracy: 0.6523 - loss: 0.5747 - val_accuracy: 0.7240 - val_loss: 0.5162
Epoch 20/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 30s 183ms/step - accuracy: 0.6519 - loss: 0.5706 - val_accuracy: 0.6529 - val_loss: 0.5484


In [ ]:
base_model_res.trainable = True
for layer in base_model_res.layers[:-10]:  # Freeze semua kecuali 10 layer terakhir
    layer.trainable = False

model_res.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history_fine_res = model_res.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=20,
    callbacks=callbacks
)


Epoch 1/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 51s 222ms/step - accuracy: 0.7102 - loss: 0.5850 - val_accuracy: 0.3606 - val_loss: 0.7041
Epoch 2/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 30s 184ms/step - accuracy: 0.7264 - loss: 0.5194 - val_accuracy: 0.6055 - val_loss: 0.6010
Epoch 3/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step - accuracy: 0.7984 - loss: 0.4558

162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 159ms/step - accuracy: 0.7985 - loss: 0.4557 - val_accuracy: 0.8085 - val_loss: 0.4430
Epoch 4/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.8211 - loss: 0.4151

162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 160ms/step - accuracy: 0.8211 - loss: 0.4150 - val_accuracy: 0.8139 - val_loss: 0.4175
Epoch 5/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 25s 156ms/step - accuracy: 0.8464 - loss: 0.3687 - val_accuracy: 0.7862 - val_loss: 0.6011
Epoch 6/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 30s 184ms/step - accuracy: 0.8698 - loss: 0.3348 - val_accuracy: 0.7991 - val_loss: 0.4257
Epoch 7/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.8703 - loss: 0.3286

162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 161ms/step - accuracy: 0.8703 - loss: 0.3286 - val_accuracy: 0.8363 - val_loss: 0.4157
Epoch 8/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.8862 - loss: 0.2924

162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 162ms/step - accuracy: 0.8862 - loss: 0.2924 - val_accuracy: 0.8336 - val_loss: 0.3874
Epoch 9/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step - accuracy: 0.8947 - loss: 0.2925

162/162 ━━━━━━━━━━━━━━━━━━━━ 32s 197ms/step - accuracy: 0.8947 - loss: 0.2925 - val_accuracy: 0.8593 - val_loss: 0.3864
Epoch 10/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 160ms/step - accuracy: 0.9095 - loss: 0.2590 - val_accuracy: 0.8322 - val_loss: 0.3879
Epoch 11/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 159ms/step - accuracy: 0.9054 - loss: 0.2596 - val_accuracy: 0.8410 - val_loss: 0.4910
Epoch 12/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 40s 155ms/step - accuracy: 0.9025 - loss: 0.2621 - val_accuracy: 0.7794 - val_loss: 0.8159
Epoch 13/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 159ms/step - accuracy: 0.9189 - loss: 0.2364 - val_accuracy: 0.8234 - val_loss: 0.5937
Epoch 14/20
162/162 ━━━━━━━━━━━━━━━━━━━━ 26s 159ms/step - accuracy: 0.8234 - loss: 0.3654 - val_accuracy: 0.7943 - val_loss: 0.7267


In [ ]:
model_res.save('model_res.h5')


In [ ]:
model_res.save('/content/drive/MyDrive/model_res.keras')


In [ ]:
import joblib
joblib.dump(model_res, 'model_res.joblib')



['model_res.joblib']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
model_res.save('/content/drive/MyDrive/model_res.h5')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
joblib.dump(model, '/content/drive/MyDrive/model.joblib')


['/content/drive/MyDrive/model.joblib']

## Fine tuning dengan turner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
def build_model(hp):
    base_model = ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=(224, 224, 3)
    )
    # Hyperparameter: jumlah layer yang di-unfreeze
    unfreeze_layers = hp.Int('unfreeze_layers', min_value=5, max_value=50, step=5)
    base_model.trainable = True
    for layer in base_model.layers[:-unfreeze_layers]:
        layer.trainable = False

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(
            hp.Int('dense_units', min_value=32, max_value=256, step=32),
            activation='relu'
        ),
        layers.Dropout(hp.Float('dropout', 0.2, 0.6, step=0.1)),
        layers.Dense(2, activation='softmax')
    ])

    lr = hp.Choice('learning_rate', [1e-4, 5e-5, 1e-5])
    model.compile(
        optimizer=optimizers.Adam(learning_rate=lr),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [ ]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=20,  # Banyak kombinasi yang dicoba
    executions_per_trial=1,
    directory='tuner_dir',
    project_name='drunk_sober_finetune'
)

callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True)
]

tuner.search(
    train_generator,
    validation_data=valid_generator,
    epochs=10,
    callbacks=callbacks
)


Trial 20 Complete [00h 09m 22s]
val_accuracy: 0.8457374572753906

Best val_accuracy So Far: 0.8863328695297241
Total elapsed time: 02h 20m 19s


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparams = tuner.get_best_hyperparameters(num_trials=1)[0]

print("Best hyperparameters:")
print(best_hyperparams.values)

# Simpan model terbaik
best_model.save('best_resnet50_tuned.h5')


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 70 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Best hyperparameters:
{'unfreeze_layers': 25, 'dense_units': 64, 'dropout': 0.30000000000000004, 'learning_rate': 5e-05}


In [ ]:
# Evaluasi model pada data uji
score = best_model.evaluate(test_generator)
print(f"Test loss: {score[0]:.4f}")
print(f"Test accuracy: {score[1]:.4f}")


23/23 ━━━━━━━━━━━━━━━━━━━━ 12s 204ms/step - accuracy: 0.8888 - loss: 0.3222
Test loss: 0.3470
Test accuracy: 0.8828


fine fine tuning coba2

In [ ]:


unfreeze_layers = 25
dense_units = 64
dropout_rate = 0.3
learning_rate = 5e-5

input_shape = (224, 224, 3)
num_classes = 2

base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=input_shape
)
base_model.trainable = True
for layer in base_model.layers[:-unfreeze_layers]:
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(dense_units, activation='relu'),
    layers.Dropout(dropout_rate),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=optimizers.Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
